# Trading Strategy & Backtesting
**Date:** February 2026  
**Objective:** To implement and backtest trading strategy for a selected stock using Python, Pandas, and Vectorized operations.

In [1]:
pip install numpy pandas yfinance matplotlib plotly ipython nbformat nbconvert

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf;

/Users/pierson.chu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Importing chosen stock data from yfinance

In [3]:
goog = yf.Ticker("GOOG")
google_five_year_history = goog.history(period='5y')

In [4]:
window = 140 #20 week
google_five_year_history['SMA'] = google_five_year_history['Close'].rolling(window=window).mean()       # Simple Moving Average
google_five_year_history['Vol'] = google_five_year_history['Close'].std()        # Rolling Volatility

In [5]:
# baseline log returns from jsut holding
google_five_year_history['Log_Ret'] = np.log(google_five_year_history['Close']/ google_five_year_history['Close'].shift(1))

In [7]:
google_five_year_history

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA,Vol,Log_Ret,H-L,H-PC,L-PC
Date,,,,,,,,,,,,,
2021-02-25 00:00:00-05:00,102.602961,103.964257,100.312144,100.811897,36568000,0.0,0.0,NaN,53.805941,NaN,3.652113,NaN,NaN
2021-02-26 00:00:00-05:00,101.762761,102.779632,100.052588,101.084846,41670000,0.0,0.0,NaN,53.805941,0.002704,2.727044,1.967735,0.759310
2021-03-01 00:00:00-05:00,102.060518,103.549351,101.543400,103.300720,28090000,0.0,0.0,NaN,53.805941,0.021684,2.005951,2.464505,0.458554
2021-03-02 00:00:00-05:00,103.036704,104.435212,102.792041,103.019333,22692000,0.0,0.0,NaN,53.805941,-0.002728,1.643171,1.134492,0.508679
2021-03-03 00:00:00-05:00,102.591035,103.648510,99.751833,100.581116,29684000,0.0,0.0,NaN,53.805941,-0.023952,3.896677,0.629177,3.267500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-18 00:00:00-05:00,302.432007,305.910004,301.980011,303.940002,15847700,0.0,0.0,275.016092,53.805941,0.003692,3.929993,3.089996,0.839996
2026-02-19 00:00:00-05:00,302.519989,306.029999,300.670013,303.559998,13448600,0.0,0.0,275.776270,53.805941,-0.001251,5.359985,2.089996,3.269989
2026-02-20 00:00:00-05:00,304.799988,316.760010,304.420013,314.899994,33663100,0.0,0.0,276.650112,53.805941,0.036676,12.339996,13.200012,0.860016


### Setting sell/purchase signal
In a momentum trading signal there are one thing to focus on: the ATR (average true range). The concept is as follows:
* Calculate the ATR (Average True Range) to measure volatility.
* Identify a "squeeze" using the ATR: When the current ATR is lower than the 30-day average ATR (it signals a consolidation period or a squeeze)
* Buy Signal: If Price breaks the Highest High of the last 20 days + 1 ATR, BUY.
* Exit Signal: If Price falls below the Lowest Low of the last 10 days, SELL.

What is an ATR?
* True Range (TR) looks at the greatest of three distances: Today's High to Today's Low, Yesterday's Close to Today's High, Yesterday's Close to Today's Low.
    * The reason for this rather than just looking at the intraday highs and lows is to avoid missing out any overnight trading effects
* ATR is simply the moving average (usually 14 days) of those True Range values.

Identifying the squeeze
$$ Current ATR<30-Day Average ATR $$

Identifying the buy signal
* The 20-Day High: This is a classic "Donchian Channel" breakout. If the price reaches a new 20-day high, it is a of a possible breakout
* The "+ 1 ATR" (The Noise Filter): By requiring the price to break the high plus one full unit of average volatility (ATR), you are ensuring the move has real power behind it.

Identifying the exit signal
* The lowest low in the past 10 days, natural the periods set are variable due to "Stock Personalities"

In [12]:
# calculate ATR
# TR = Max(High-Low, |High-PrevClose|, |Low-PrevClose|)

google_five_year_history['H-L'] = google_five_year_history['High'] - google_five_year_history['Low']
google_five_year_history['H-PC'] = abs(google_five_year_history['High'] - google_five_year_history['Close'].shift(1))
google_five_year_history['L-PC'] = abs(google_five_year_history['Low'] - google_five_year_history['Close'].shift(1))
google_five_year_history['TR'] = google_five_year_history[['H-L', 'H-PC', 'L-PC']].max(axis=1)
google_five_year_history['ATR'] = google_five_year_history['TR'].rolling(window=14).mean()

In [13]:
google_five_year_history

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA,Vol,Log_Ret,H-L,H-PC,L-PC,TR,ATR
Date,,,,,,,,,,,,,,,
2021-02-25 00:00:00-05:00,102.602961,103.964257,100.312144,100.811897,36568000,0.0,0.0,NaN,53.805941,NaN,3.652113,NaN,NaN,3.652113,NaN
2021-02-26 00:00:00-05:00,101.762761,102.779632,100.052588,101.084846,41670000,0.0,0.0,NaN,53.805941,0.002704,2.727044,1.967735,0.759310,2.727044,NaN
2021-03-01 00:00:00-05:00,102.060518,103.549351,101.543400,103.300720,28090000,0.0,0.0,NaN,53.805941,0.021684,2.005951,2.464505,0.458554,2.464505,NaN
2021-03-02 00:00:00-05:00,103.036704,104.435212,102.792041,103.019333,22692000,0.0,0.0,NaN,53.805941,-0.002728,1.643171,1.134492,0.508679,1.643171,NaN
2021-03-03 00:00:00-05:00,102.591035,103.648510,99.751833,100.581116,29684000,0.0,0.0,NaN,53.805941,-0.023952,3.896677,0.629177,3.267500,3.896677,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-18 00:00:00-05:00,302.432007,305.910004,301.980011,303.940002,15847700,0.0,0.0,275.016092,53.805941,0.003692,3.929993,3.089996,0.839996,3.929993,10.997140
2026-02-19 00:00:00-05:00,302.519989,306.029999,300.670013,303.559998,13448600,0.0,0.0,275.776270,53.805941,-0.001251,5.359985,2.089996,3.269989,5.359985,10.267853
2026-02-20 00:00:00-05:00,304.799988,316.760010,304.420013,314.899994,33663100,0.0,0.0,276.650112,53.805941,0.036676,12.339996,13.200012,0.860016,13.200012,10.664283
